In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/trained_final/other/v2/1/v_2bert_sentiment_model_final_on_Twitter.bin
/kaggle/input/fb-data/final_facebookfile.csv


In [3]:
!pip install bertopic
!pip install --upgrade nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 56.4 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl.metadata (3.1 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 9.6 MB/s eta 0:00:00
Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=819466 sha256=7080d7e6629734423b5b9befd81f4c06f08d0a9c31999ae7d483836a297946e6
  Stored in directory: /root/.cache/pip/wheels/75/0b/3b/dc4f60b7cc455efaefb62883a7483e76f09d06ca81cf87d610
Successfully built hdbscan
  Attempting uninstall: cython
    Found existing installation: Cytho

In [34]:
import pandas as pd

df = pd.read_csv('/kaggle/input/fb-data/final_facebookfile.csv')  # replace 'your_data.csv' with the path to your file

In [37]:
df['unique_id'] = range(len(df))

df['Post'] = df['Post'].str.lower()


In [38]:
df = df.dropna(subset=['Post'])

In [39]:
df.shape

(6596, 5)

In [40]:
df.head(3)

,Unnamed: 0,Post,year,keyword,unique_id
0,0,hydrogel is the key ingredient which enabled t...,2020,health_biosensers,0
1,1,jesus chastised the religious elite of his day...,2020,health_biosensers,1
2,2,frances frame is owning her power!\n.\ni love ...,2020,health_biosensers,2


In [41]:
df_copy = df.copy()

In [42]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import re

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [47]:
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

def preprocess_text(text):
    # Check if the text contains only ASCII characters
    if all(ord(char) < 128 for char in text):
        # Convert text to lowercase
        text = text.lower()
        # Remove URLs
        text = re.sub(r'http\S+|www.\S+', '', text)
        # Remove special characters, numbers, and punctuations
        text = ''.join([char for char in text if char.isalnum() or char.isspace()])
        # Remove specific terms like 'url', 'images', and 'thumbnail'
        text = re.sub(r'\b(url|images|thumbnail)\b', '', text)
        return text
    else:
        return ''  # Return empty string for non-English text

# Tokenization, Stopwords Removal, and Lemmatization
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text):
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and lemmatize
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token.lower() not in stop_words]
    return ' '.join(tokens)

# Apply preprocessing to the entire dataset
# Assuming df_copy is defined somewhere in your code
df_copy['Post'] = df_copy['Post'].apply(preprocess_text)
df_copy['Post'] = df_copy['Post'].apply(tokenize_and_lemmatize)


In [48]:
df_copy.shape

(6596, 5)

In [54]:
# Drop rows with an empty string in the 'Post' column
df_copy.dropna(subset=['Post'], inplace=True)


In [55]:
df_copy.shape

(6596, 5)

In [56]:
# Assuming df is your DataFrame with a 'text' column
df_copy['Post'] = df_copy['Post'].astype(str)  # Convert to string type to ensure consistency

# Remove duplicates from the 'text' column
df_copy.drop_duplicates(subset='Post', inplace=True)

# Now df contains unique values in the 'text' column

In [57]:
df_copy.shape

(5391, 5)

In [58]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer

# Specify the path where you saved the model and optimizer state
load_path = "/kaggle/input/trained_final/other/v2/1/v_2bert_sentiment_model_final_on_Twitter.bin"

# Initialize the model and optimizer architecture
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

# Load the model and optimizer states
checkpoint = torch.load(load_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

# Other components
class_weights_tensor = checkpoint['class_weights']
tokenizer = checkpoint['tokenizer']
max_length = checkpoint['max_length']

# Move the model to the appropriate device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [59]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import DataLoader, TensorDataset
from bertopic import BERTopic
import pandas as pd

# Load the pre-trained BERT model and tokenizer
load_path = "/kaggle/input/trained_final/other/v2/1/v_2bert_sentiment_model_final_on_Twitter.bin"
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
checkpoint = torch.load(load_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Read the data
# df = pd.read_csv("D:\\Desktop Files\\FB_SCRAPES\\Reddit\\CODED_RAW_coder_validation_reddit_10_percent.csv")

# Drop unrelated rows and remove null values
# df_copy = df_copy[~df_copy['Unnamed: 2'].str.contains('unrelated', na=False)]
df_copy = df_copy.dropna(subset=['Post'])
# df_copy = df_copy.drop('Unnamed: 2', axis=1)

def perform_sentiment_analysis(texts, model, tokenizer, batch_size=4):
    # Tokenize and convert to tensors
    predictions = []
    num_texts = len(texts)
    for i in range(0, num_texts, batch_size):
        batch_texts = texts[i:i+batch_size]
        batch_texts = [str(text) for text in batch_texts]  # Ensure each element is a string
        tokens = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt")
        inputs = {key: tokens[key].to(device) for key in tokens}

        # Perform inference using the loaded model
        model.eval()
        with torch.no_grad():
            outputs = model(**inputs)
            _, preds = torch.max(outputs.logits, 1)
            predictions.extend(preds.cpu().numpy())

    return predictions


# Specify batch size
batch_size = 2  # Adjust as needed

df_copy['predicted_sentiment'] = perform_sentiment_analysis(df_copy['Post'], model, tokenizer, batch_size=batch_size)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [60]:
df_copy.head(5)

,Unnamed: 0,Post,year,keyword,unique_id,predicted_sentiment
0,0,hydrogel key ingredient enabled torture human ...,2020,health_biosensers,0,2
1,1,jesus chastised religious elite day able see s...,2020,health_biosensers,1,2
2,2,france frame owning power love friend also cli...,2020,health_biosensers,2,2
3,3,would like share detail achieving vyvo thanks ...,2020,health_biosensers,3,2
4,4,thing amazing seriously im amazed technology i...,2020,health_biosensers,4,2


In [61]:
df_copy['predicted_sentiment_category'] = df_copy['predicted_sentiment'].map({0: 'negative', 1: 'neutral', 2: 'positive'})

In [62]:
df_copy.shape

(5391, 7)

In [72]:
import random

# Set a random seed
random.seed(42)  # You can use any integer value here

# Perform topic modeling without sentiment analysis
# Initialize BERTopic model
topic_model = BERTopic(language="english", nr_topics= 10, calculate_probabilities=True, n_gram_range=(1, 1))

# Fit BERTopic model to preprocessed text data
texts = df_copy['Post'].tolist()
topics, probabilities = topic_model.fit_transform(texts)

# Add main words for each topic as a new column
main_words_list = [topic_model.get_topic(topic) for topic in topics]
words_only_list = [[word[0] for word in main_words] for main_words in main_words_list]
df_copy['main_words_only'] = words_only_list

# Assign topic labels to documents
df_copy['topic_label'] = topics

# Determine dominant sentiment for each topic label
dominant_sentiment_per_topic = {}
for topic_id in set(topics):
    topic_indices = df_copy[df_copy['topic_label'] == topic_id].index
    topic_sentiments = df_copy.loc[topic_indices, 'predicted_sentiment']
    sentiment_counts = topic_sentiments.value_counts()
    if len(sentiment_counts) >= 2:
        dominant_sentiment = sentiment_counts.index[0]  # Get the most frequent sentiment for the topic
        second_dominant_sentiment = sentiment_counts.index[1]  # Get the second most frequent sentiment for the topic
    else:
        dominant_sentiment = sentiment_counts.index[0]
        second_dominant_sentiment = None
    dominant_sentiment_per_topic[topic_id] = (dominant_sentiment, second_dominant_sentiment)

# Map dominant sentiment to each topic label
df_copy['dominant_sentiment'], df_copy['second_dominant_sentiment'] = zip(*df_copy['topic_label'].map(dominant_sentiment_per_topic))

# Display the DataFrame
print(df_copy)

      Unnamed: 0                                               Post  year  \
0              0  hydrogel key ingredient enabled torture human ...  2020   
1              1  jesus chastised religious elite day able see s...  2020   
2              2  france frame owning power love friend also cli...  2020   
3              3  would like share detail achieving vyvo thanks ...  2020   
4              4  thing amazing seriously im amazed technology i...  2020   
...          ...                                                ...   ...   
6597        6597  biodegradable sensor monitor level pesticide v...  2024   
6600        6600  much necessary sleep professional athlete slee...  2024   
6601        6601  movement detective motion tracking physiothera...  2024   
6602        6602  sg micro operational amplifier sgm8634 low noi...  2024   
6603        6603  happy international woman day today every day ...  2024   

                keyword  unique_id  predicted_sentiment  \
0     health_bio

In [73]:
# Map predictions to the new column
df_copy['dominant_sentiment_category'] = df_copy['dominant_sentiment'].map({0: 'negative', 1: 'neutral', 2: 'positive'})

df_copy['second_dominant_sentiment_category'] = df_copy['second_dominant_sentiment'].map({0: 'negative', 1: 'neutral', 2: 'positive'})

In [74]:
df_copy.columns

Index(['Unnamed: 0', 'Post', 'year', 'keyword', 'unique_id',
       'predicted_sentiment', 'predicted_sentiment_category',
       'main_words_only', 'topic_label', 'dominant_sentiment',
       'second_dominant_sentiment', 'dominant_sentiment_category',
       'second_dominant_sentiment_category'],
      dtype='object')

In [75]:
# If you want to merge the original and preprocessed DataFrames later
merged_df = pd.merge(df, df_copy[['Post', 'year','keyword','unique_id','predicted_sentiment','predicted_sentiment_category','main_words_only','topic_label','dominant_sentiment','second_dominant_sentiment']], on='unique_id', how='inner')

# Now, merged_df contains both 'text_x' and 'text_y' columns
# You can access them as merged_df['text_x'] and merged_df['text_y']

# Drop the preprocessed text column
#merged_df.drop('Unnamed: 0', axis=1, inplace=True)



# Rename the original text column for clarity
merged_df.rename(columns={'Post_x': 'original_post'}, inplace=True)

In [76]:
merged_df.head(5)

,Unnamed: 0,original_post,year_x,keyword_x,unique_id,Post_y,year_y,keyword_y,predicted_sentiment,predicted_sentiment_category,main_words_only,topic_label,dominant_sentiment,second_dominant_sentiment
0,0,hydrogel is the key ingredient which enabled t...,2020,health_biosensers,0,hydrogel key ingredient enabled torture human ...,2020,health_biosensers,2,positive,"[wearable, see, health, sensor, medical, techn...",-1,2,1
1,1,jesus chastised the religious elite of his day...,2020,health_biosensers,1,jesus chastised religious elite day able see s...,2020,health_biosensers,2,positive,"[wearable, see, health, sensor, medical, techn...",-1,2,1
2,2,frances frame is owning her power!\n.\ni love ...,2020,health_biosensers,2,france frame owning power love friend also cli...,2020,health_biosensers,2,positive,"[wearable, see, health, sensor, medical, techn...",-1,2,1
3,3,i would like to share with all of you some det...,2020,health_biosensers,3,would like share detail achieving vyvo thanks ...,2020,health_biosensers,2,positive,"[wearable, see, health, sensor, medical, techn...",-1,2,1
4,4,"this thing is amazing! \nseriously, i’m so ama...",2020,health_biosensers,4,thing amazing seriously im amazed technology i...,2020,health_biosensers,2,positive,"[wearable, sensor, see, health, medical, watch...",0,2,1


In [77]:
# Save the DataFrame to a CSV file
merged_df.to_csv('/kaggle/working/Facebook_BERT_FINAL.csv', index=False)

In [78]:
# Assuming df_copy contains your DataFrame
processed_texts = df_copy['Post'].tolist()

In [79]:
topic_model.visualize_barchart()

In [80]:
topic_model.get_topics()

{-1: [('wearable', 0.04126008330606888),
  ('see', 0.036576994787935566),
  ('health', 0.033138721149882824),
  ('sensor', 0.031247635601807607),
  ('medical', 0.02399490029464223),
  ('technology', 0.021258363610262853),
  ('device', 0.018961899828157786),
  ('new', 0.016381204412835672),
  ('data', 0.015796785585338917),
  ('healthcare', 0.015069303676012382)],
 0: [('wearable', 0.04171012468493975),
  ('sensor', 0.037761776510220564),
  ('see', 0.03558852913914465),
  ('health', 0.03313137164846891),
  ('medical', 0.022311684464069025),
  ('watch', 0.02091047473359119),
  ('device', 0.02054230570448239),
  ('technology', 0.019145505493198878),
  ('smart', 0.017483920170446855),
  ('new', 0.016847429646386698)],
 1: [('see', 0.05261684488782029),
  ('expo', 0.043576694579090344),
  ('india', 0.04051376093278227),
  ('mobile', 0.03688613468840359),
  ('2022', 0.02624629427658104),
  ('dti', 0.02557169281393571),
  ('hiraya', 0.02441232705188244),
  ('regional', 0.02172474427901237),
 

In [81]:
topic_model.visualize_distribution(topic_model.probabilities_[0])

In [82]:
topic_model.visualize_topics()

In [51]:
merged_df.shape

(5946, 14)

In [26]:
from gensim import corpora

# Assuming processed_texts is a list of strings
# Split each string into tokens
processed_texts_tokenized = [text.split() for text in processed_texts]

# Create dictionary
dictionary = corpora.Dictionary(processed_texts_tokenized)

# Create corpus
corpus = [dictionary.doc2bow(text) for text in processed_texts_tokenized]


In [28]:
import gensim
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel# List to store coherence scores

coherence_scores = []

# Iterate over different numbers of topics
for num_topics in range(2, 20):  # Adjust the range as needed
    # Train LDA model
    lda_model = gensim.models.LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
    
    # Compute coherence score
    coherence_model_lda = CoherenceModel(model=lda_model, texts=processed_texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model_lda.get_coherence()
    coherence_scores.append((num_topics, coherence_score))

# Find the number of topics with the highest coherence score
best_num_topics, best_coherence_score = max(coherence_scores, key=lambda x: x[1])

print("Best number of topics:", best_num_topics)
print("Coherence score:", best_coherence_score)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/opt/conda/lib/python3.10/si

Best number of topics: 2
Coherence score: nan


In [33]:
# Save the DataFrame to a CSV file
merged_df.to_csv('/kaggle/working/Facebook_BERT_FINAL.csv', index=False)